In [ ]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset path
dataset_path = "/content/drive/MyDrive/FYP models/preprocessing/fyp_desease"

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Print class-to-index mapping
print("📂 Class to Index Mapping:")
for class_name, class_idx in full_dataset.class_to_idx.items():
    print(f"  {class_idx}: {class_name}")

# Split into train/validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"\n📊 Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

# Load pretrained ResNet50
model = models.resnet50(pretrained=True)
num_classes = 3
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Freeze all layers except last block + fc
for param in model.parameters():
    param.requires_grad = False
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Early stopping
best_val_loss = float('inf')
patience = 5
patience_counter = 0

# Training
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    all_preds, all_labels = [], []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(all_labels, all_preds)
    scheduler.step()

    # Validation
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)

    print(f"📅 Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {running_loss/len(train_loader):.4f} | "
          f"Train Acc: {train_acc:.4f} | "
          f"Val Acc: {val_acc:.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        save_dir = "/content/drive/MyDrive/FYP models/models/resnet50(3_fullchicken)"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, "resnet50_3class_best_full.pth")
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model saved (epoch {epoch+1})")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

print("🏁 Training complete.")


📂 Class to Index Mapping:
  0: 1_Marek
  1: 2_desease
  2: 3_healthy

📊 Training samples: 1207, Validation samples: 302
📅 Epoch [1/30] | Train Loss: 0.6240 | Train Acc: 0.7258 | Val Acc: 0.8013
✅ Model saved (epoch 1)
📅 Epoch [2/30] | Train Loss: 0.3255 | Train Acc: 0.8906 | Val Acc: 0.8411
✅ Model saved (epoch 2)
📅 Epoch [3/30] | Train Loss: 0.1952 | Train Acc: 0.9370 | Val Acc: 0.8709
✅ Model saved (epoch 3)
📅 Epoch [4/30] | Train Loss: 0.1345 | Train Acc: 0.9553 | Val Acc: 0.8775
✅ Model saved (epoch 4)
📅 Epoch [5/30] | Train Loss: 0.1345 | Train Acc: 0.9544 | Val Acc: 0.8808
📅 Epoch [6/30] | Train Loss: 0.0907 | Train Acc: 0.9677 | Val Acc: 0.8907
📅 Epoch [7/30] | Train Loss: 0.0728 | Train Acc: 0.9793 | Val Acc: 0.8841
📅 Epoch [8/30] | Train Loss: 0.0715 | Train Acc: 0.9760 | Val Acc: 0.8775
📅 Epoch [9/30] | Train Loss: 0.0495 | Train Acc: 0.9793 | Val Acc: 0.8841
⏹️ Early stopping triggered.
🏁 Training complete.


In [ ]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset path
dataset_path = "/content/drive/MyDrive/FYP models/preprocessing/fyp_specific_desease"

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Print class-to-index mapping
print("📂 Class to Index Mapping:")
for class_name, class_idx in full_dataset.class_to_idx.items():
    print(f"  {class_idx}: {class_name}")

# Split into train/validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"\n📊 Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

# Load pretrained ResNet50
model = models.resnet50(pretrained=True)
num_classes = 3
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Freeze all layers except last block + fc
for param in model.parameters():
    param.requires_grad = False
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Early stopping
best_val_loss = float('inf')
patience = 5
patience_counter = 0

# Training
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    all_preds, all_labels = [], []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(all_labels, all_preds)
    scheduler.step()

    # Validation
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)

    print(f"📅 Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {running_loss/len(train_loader):.4f} | "
          f"Train Acc: {train_acc:.4f} | "
          f"Val Acc: {val_acc:.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        save_dir = "/content/drive/MyDrive/FYP models/models/resnet50(3_neck)"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, "resnet50_3class_best_neck.pth")
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model saved (epoch {epoch+1})")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

print("🏁 Training complete.")


📂 Class to Index Mapping:
  0: 1_infectious_coryza
  1: 2_fowl_pox
  2: 3_healthy

📊 Training samples: 339, Validation samples: 85


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


📅 Epoch [1/30] | Train Loss: 0.7881 | Train Acc: 0.6549 | Val Acc: 0.7059
✅ Model saved (epoch 1)
📅 Epoch [2/30] | Train Loss: 0.2826 | Train Acc: 0.9145 | Val Acc: 0.9294
✅ Model saved (epoch 2)
📅 Epoch [3/30] | Train Loss: 0.1435 | Train Acc: 0.9499 | Val Acc: 0.9059
✅ Model saved (epoch 3)
📅 Epoch [4/30] | Train Loss: 0.0561 | Train Acc: 0.9941 | Val Acc: 0.9059
📅 Epoch [5/30] | Train Loss: 0.0301 | Train Acc: 0.9941 | Val Acc: 0.9059
📅 Epoch [6/30] | Train Loss: 0.0232 | Train Acc: 0.9971 | Val Acc: 0.9059
✅ Model saved (epoch 6)
📅 Epoch [7/30] | Train Loss: 0.0180 | Train Acc: 0.9971 | Val Acc: 0.9294
✅ Model saved (epoch 7)
📅 Epoch [8/30] | Train Loss: 0.0141 | Train Acc: 0.9971 | Val Acc: 0.9294
📅 Epoch [9/30] | Train Loss: 0.0102 | Train Acc: 1.0000 | Val Acc: 0.9176
📅 Epoch [10/30] | Train Loss: 0.0063 | Train Acc: 1.0000 | Val Acc: 0.9176
📅 Epoch [11/30] | Train Loss: 0.0091 | Train Acc: 1.0000 | Val Acc: 0.9176
📅 Epoch [12/30] | Train Loss: 0.0156 | Train Acc: 0.9971 | Val A

In [ ]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset path
dataset_path = "/content/drive/MyDrive/FYP models/preprocessing/fyp_specific_desease"

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Or (120, 120) if images are very small
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Print class-to-index mapping
print("📂 Class to Index Mapping:")
for class_name, class_idx in full_dataset.class_to_idx.items():
    print(f"  {class_idx}: {class_name}")

# Split into train/validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"\n📊 Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

# Load pretrained MobileNetV3 Small
model = models.mobilenet_v3_small(pretrained=True)
num_classes = 3
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model.to(device)

# Freeze all layers except classifier
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Early stopping
best_val_loss = float('inf')
patience = 5
patience_counter = 0

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    all_preds, all_labels = [], []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(all_labels, all_preds)
    scheduler.step()

    # Validation
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)

    print(f"📅 Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {running_loss/len(train_loader):.4f} | "
          f"Train Acc: {train_acc:.4f} | "
          f"Val Acc: {val_acc:.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        save_dir = "/content/drive/MyDrive/FYP models/models/mobilenetv3(3_neck)"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, "mobilenetv3_3class_best_neck.pth")
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model saved (epoch {epoch+1})")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

print("🏁 Training complete.")


📂 Class to Index Mapping:
  0: 1_infectious_coryza
  1: 2_fowl_pox
  2: 3_healthy

📊 Training samples: 339, Validation samples: 85


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


📅 Epoch [1/30] | Train Loss: 1.0407 | Train Acc: 0.4956 | Val Acc: 0.4588
✅ Model saved (epoch 1)
📅 Epoch [2/30] | Train Loss: 0.9102 | Train Acc: 0.6431 | Val Acc: 0.4941
✅ Model saved (epoch 2)
📅 Epoch [3/30] | Train Loss: 0.8222 | Train Acc: 0.6873 | Val Acc: 0.5294
✅ Model saved (epoch 3)
📅 Epoch [4/30] | Train Loss: 0.7693 | Train Acc: 0.7050 | Val Acc: 0.5412
✅ Model saved (epoch 4)
📅 Epoch [5/30] | Train Loss: 0.7050 | Train Acc: 0.7699 | Val Acc: 0.5647
✅ Model saved (epoch 5)
📅 Epoch [6/30] | Train Loss: 0.6699 | Train Acc: 0.7906 | Val Acc: 0.5765
✅ Model saved (epoch 6)
📅 Epoch [7/30] | Train Loss: 0.6375 | Train Acc: 0.7906 | Val Acc: 0.6353
✅ Model saved (epoch 7)
📅 Epoch [8/30] | Train Loss: 0.6394 | Train Acc: 0.7640 | Val Acc: 0.6471
✅ Model saved (epoch 8)
📅 Epoch [9/30] | Train Loss: 0.5947 | Train Acc: 0.8142 | Val Acc: 0.6353
✅ Model saved (epoch 9)
📅 Epoch [10/30] | Train Loss: 0.5597 | Train Acc: 0.8112 | Val Acc: 0.7059
✅ Model saved (epoch 10)
📅 Epoch [11/30] | 

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from torchvision.models import efficientnet_b0
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset path
dataset_path = "/content/drive/MyDrive/FYP models/preprocessing/fyp_specific_desease"

# Transform
transform = transforms.Compose([
    transforms.Resize((150, 150)),  # or use (150, 150)
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Dataset loading
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Class mapping
print("📂 Class to Index Mapping:")
for class_name, class_idx in full_dataset.class_to_idx.items():
    print(f"  {class_idx}: {class_name}")

# Train-validation split
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"\n📊 Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

# Load EfficientNet
model = efficientnet_b0(pretrained=True)
num_classes = 3
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model.to(device)

# Freeze all except last block
for param in model.parameters():
    param.requires_grad = False
for param in model.features[-1].parameters():
    param.requires_grad = True
for param in model.classifier.parameters():
    param.requires_grad = True

# Loss, optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Early stopping
best_val_loss = float('inf')
patience = 5
patience_counter = 0

# Training
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    all_preds, all_labels = [], []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(all_labels, all_preds)
    scheduler.step()

    # Validation
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)

    print(f"📅 Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {running_loss/len(train_loader):.4f} | "
          f"Train Acc: {train_acc:.4f} | "
          f"Val Acc: {val_acc:.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        save_dir = "/content/drive/MyDrive/FYP models/models/efficientnet_b0_neck"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, "efficientnet_b0_3class_best_neck.pth")
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model saved (epoch {epoch+1})")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

print("🏁 Training complete.")


📂 Class to Index Mapping:
  0: 1_infectious_coryza
  1: 2_fowl_pox
  2: 3_healthy

📊 Training samples: 339, Validation samples: 85


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


📅 Epoch [1/30] | Train Loss: 1.1159 | Train Acc: 0.3540 | Val Acc: 0.5059
✅ Model saved (epoch 1)
📅 Epoch [2/30] | Train Loss: 1.0157 | Train Acc: 0.5133 | Val Acc: 0.5059
✅ Model saved (epoch 2)
📅 Epoch [3/30] | Train Loss: 0.9488 | Train Acc: 0.6372 | Val Acc: 0.6353
✅ Model saved (epoch 3)
📅 Epoch [4/30] | Train Loss: 0.9060 | Train Acc: 0.6726 | Val Acc: 0.6471
✅ Model saved (epoch 4)
📅 Epoch [5/30] | Train Loss: 0.8313 | Train Acc: 0.7257 | Val Acc: 0.7176
✅ Model saved (epoch 5)
📅 Epoch [6/30] | Train Loss: 0.7981 | Train Acc: 0.7670 | Val Acc: 0.6706
✅ Model saved (epoch 6)
📅 Epoch [7/30] | Train Loss: 0.7790 | Train Acc: 0.7994 | Val Acc: 0.6824
✅ Model saved (epoch 7)
📅 Epoch [8/30] | Train Loss: 0.7561 | Train Acc: 0.7994 | Val Acc: 0.7765
✅ Model saved (epoch 8)
📅 Epoch [9/30] | Train Loss: 0.7402 | Train Acc: 0.7965 | Val Acc: 0.7059
📅 Epoch [10/30] | Train Loss: 0.7151 | Train Acc: 0.8053 | Val Acc: 0.7059
📅 Epoch [11/30] | Train Loss: 0.6873 | Train Acc: 0.8260 | Val Acc: